In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

In [0]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_labels[0])
plt.imshow(train_images[0], cmap="Greys_r")

data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 28, 28, 1]).astype(np.float32) / 255, train_labels.astype(np.int32)))
data = data.shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 28, 28, 1]).astype(np.float32) / 255, test_labels.astype(np.int32))).batch(128)

In [0]:
train_steps = 500


# example: two (basic) inception blocks
# TODO: add 1x1 convs ;)
inp = tf.keras.layers.Input((28, 28, 1))


conv1_1 = tf.keras.layers.Conv2D(32, 1, activation=tf.nn.relu, padding="same")(inp)
conv1_3 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(inp)
conv1_5 = tf.keras.layers.Conv2D(32, 5, activation=tf.nn.relu, padding="same")(inp)
pool1 = tf.keras.layers.MaxPool2D(3, 1, padding="same")(inp)

conc1 = tf.keras.layers.concatenate([conv1_1, conv1_3, conv1_5, pool1])


conv2_1 = tf.keras.layers.Conv2D(32, 1, activation=tf.nn.relu, padding="same")(conc1)
conv2_3 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conc1)
conv2_5 = tf.keras.layers.Conv2D(32, 5, activation=tf.nn.relu, padding="same")(conc1)
pool2 = tf.keras.layers.MaxPool2D(3, 1, padding="same")(conc1)

conc2 = tf.keras.layers.concatenate([conv2_1, conv2_3, conv2_5, pool2])

flat = tf.keras.layers.Flatten()(conc2)
out = tf.keras.layers.Dense(10)(flat)

model = tf.keras.Model(inp, out)


opt = tf.optimizers.Adam()

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.metrics.SparseCategoricalAccuracy()

In [0]:
# basic training loops can be done like this
model.compile(optimizer=opt, loss=loss_fn, metrics=[train_acc_metric])

In [0]:
model.fit(data, steps_per_epoch=1000, epochs=2)

In [0]:
model.evaluate(test_data)

In [0]:
logits_on_test_set = model.predict(test_data)

In [0]:
train_steps = 500


# example: a residual block
inp = tf.keras.layers.Input((28, 28, 1))

initial_conv = tf.keras.layers.Conv2D(32, 5, activation=tf.nn.relu, padding="same")(inp)

conv1_1 = tf.keras.layers.Conv2D(32, 5, activation=tf.nn.relu, padding="same")(initial_conv)
conv1_2 = tf.keras.layers.Conv2D(32, 5, activation=tf.nn.relu, padding="same")(conv1_1)

out1 = conv1_2 + initial_conv


flat = tf.keras.layers.Flatten()(out1)
out = tf.keras.layers.Dense(10)(flat)

model = tf.keras.Model(inp, out)


opt = tf.optimizers.Adam()

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.metrics.SparseCategoricalAccuracy()

In [0]:
# stereotypical train-step-with-function-annotation

@tf.function
def train_step(imgs, lbls):
    with tf.GradientTape() as tape:
        logits = model(imgs)
        xent = loss_fn(lbls, logits)

    varis = model.trainable_variables
    grads = tape.gradient(xent, varis)
    opt.apply_gradients(zip(grads, varis))

    return xent, logits


In [0]:
start = time.time()
for step, (img_batch, lbl_batch) in enumerate(data):
    if step > train_steps:
        break

    xent, logits = train_step(img_batch, lbl_batch)

    if not step % 100:
        train_acc_metric(lbl_batch, logits)
        acc = train_acc_metric.result()
        print("Loss: {} Accuracy: {}".format(xent, acc))
        train_acc_metric.reset_states()

        stop = time.time()
        print("took {} seconds\n".format(stop-start))
        start = time.time()


In [0]:
test_acc_metric = tf.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
    test_acc_metric(lbl_batch, model(img_batch))

test_acc_metric.result()